In [1]:
import pandas as pd
import numpy as np
import vectorbt as vbt

# 创建示例数据
ag_df = pd.read_excel('data/ag_df.xlsx')
ag_df = ag_df[ag_df["settle"] != 0]
ag_df.drop("Unnamed: 0", axis=1, inplace=True)
ag_df=ag_df[["date","buy_sell_rate"]]
ag_df

,date,buy_sell_rate
0,2012-05-10,0.260501
1,2012-05-11,-0.899700
2,2012-05-14,-0.368078
3,2012-05-15,0.513435
4,2012-05-16,-1.009386
...,...,...
3005,2024-09-09,-0.788304
3006,2024-09-10,0.666194
3007,2024-09-11,2.034276
3008,2024-09-12,0.684644


In [3]:
# from numba import njit
# @njit
def strategy(df,buy=1.3,sell=-1):

    df['op'] = df['buy_sell_rate'].apply(lambda x: 'buy' if x > buy else ('sell' if x < sell else None))
    filtered_df = df[df['op'].notnull()]
    
    # 获取买卖点，并且删除没有买卖点的数据
    buy_df = filtered_df[filtered_df['op'] == 'buy']
    sell_df = filtered_df[filtered_df['op'] == 'sell']
    sell_df.reset_index(drop=True, inplace=True)
    buy_df.reset_index(drop=True, inplace=True)
    # 排序
    sell_dates= sell_df['date']
    sell_dates.sort_values()
    
    # 计算卖出日期
    def cal_sell_date(buy_date):
        # Filter the DataFrame first
        filtered_dates = sell_df[sell_dates >= buy_date + pd.Timedelta(days=30) and sell_dates<= buy_date + pd.Timedelta(days=30)]['date'].values
        
        # Check if the filtered array is not empty
        if filtered_dates.size > 0:
            return filtered_dates[0]
        else:
            return None  # or handle the case where no valid sell date is found
    
    # 为buy_df添加 sell_date 列
    buy_df['sell_date'] = buy_df['date'].apply(cal_sell_date)
    
    return buy_df


filtered_df =strategy(ag_df)

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1m[1m[1mNo implementation of function Function(datetime64[]) found for signature:
 
 >>> <unknown function>(Literal[str](NaT))
 
There are 2 candidate implementations:
[1m - Of which 1 did not match due to:
 Overload in function 'make_callable_template.<locals>.generic': File: numba\core\typing\templates.py: Line 174.
   With argument(s): '(unicode_type)':[0m
[1m  Rejected as the implementation raised a specific error:
    TypingError: [1mCasting unicode_type to datetime64[] directly is unsupported.[0m[0m
  raised from D:\py_env\python311-0912\python311\Lib\site-packages\numba\core\typing\builtins.py:851
[1m - Of which 1 did not match due to:
 Overload in function 'make_callable_template.<locals>.generic': File: numba\core\typing\templates.py: Line 174.
   With argument(s): '(Literal[str](NaT))':[0m
[1m  Rejected as the implementation raised a specific error:
    TypingError: [1mCasting Literal[str](N

In [41]:
def calculate_profit(df,buy=1.3,sell=-1):
    filtered_df =strategy(df,buy=buy,sell=sell)
    filtered_df=filtered_df[filtered_df["sell_date"].notna()]
    num_rows = len(filtered_df.index)
    profit = num_rows*(buy-sell-0.5)
    return profit,filtered_df

In [44]:
profits =pd.DataFrame()
for buy_pr in np.arange(0.3, 4.0, 0.1):
    for sell_pr in np.arange(-0.3, -4.0, -0.1):
        
        profit,_ = calculate_profit(ag_df,buy=buy_pr,sell=sell_pr)
        one = pd.DataFrame([buy_pr, sell_pr, profit], index=['buy_pr', 'sell_pr', 'profit'])
        profits = pd.concat([profits, one.T], axis=0)
profits
        

C:\Users\1042133624\AppData\Local\Temp\1\ipykernel_6192\886816436.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  buy_df['sell_date'] = buy_df['date'].apply(cal_sell_date)
C:\Users\1042133624\AppData\Local\Temp\1\ipykernel_6192\886816436.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  buy_df['sell_date'] = buy_df['date'].apply(cal_sell_date)
C:\Users\1042133624\AppData\Local\Temp\1\ipykernel_6192\886816436.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Data

,buy_pr,sell_pr,profit
0,0.3,-0.3,67.5
0,0.3,-0.4,135.0
0,0.3,-0.5,202.5
0,0.3,-0.6,270.0
0,0.3,-0.7,337.5
...,...,...,...
0,3.9,-3.5,0.0
0,3.9,-3.6,0.0
0,3.9,-3.7,0.0
0,3.9,-3.8,0.0


In [42]:
profit,filtered_df = calculate_profit(ag_df,buy=0.3,sell=-3.9)
filtered_df

,date,buy_sell_rate,op,sell_date


In [43]:
profit

0.0